In [ ]:
from ctraptools.kymos.detect import Detector

import ctraptools.kymos.io as kio
import ctraptools.kymos.stepfinding as ksf
import matplotlib.pyplot as plt
import numpy as np
import os
from lumicks import pylake

In [ ]:
# Parameters
path = "C:\\Users\\steph\\Documents\\People\\CTrapTools\\"
channel = 0

In [ ]:
# Loading image and calculating paths
for f in os.listdir(path):
    name = os.path.basename(os.path.splitext(f)[0])
    if os.path.splitext(f)[1] == ".tif":
        image = kio.read_image(path+f,channel,x_range=None)
    elif os.path.splitext(f)[1] == ".h5" or os.path.splitext(f)[1] == ".hdf5":
        file = pylake.File(path+f)
        kymo = next(iter(file.kymos.values()))
        image = kymo.get_image()[:,:,channel]
    else:
        continue
        
    # Initialising Detector
    detector = Detector(half_t_w=5,peak_det_thresh = 2, n_max=10, c_ub=5, c_def=3)

    # Detect peaks and track
    tracks = detector.detect(image)

    # Measuring intensity
    for track in tracks.values():
        track.measure_intensity(image)
    
    # # Step fitting (to be added later)
    # ksf.fit_steps(tracks,image)

    # Saving files
    kio.save_overlay(tracks,image,path+name+'_C'+str(channel)+"_overlay")
    kio.write_traces(tracks,path+name+'_C'+str(channel)+"_traces")
    kio.save_plots(tracks,path+name+'_C'+str(channel)+"_traces")
    # kio.write_change_points(tracks,path+name+'_C'+str(channel)+'_steps')